<br>

####   Bucket hallucination!

In [70]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)
import seaborn as sns
import matplotlib.pyplot as plt

In [71]:
# let's read in our digits data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
filename = './data/equalratio.csv'
df = pd.read_csv(filename, header=0)   # encoding="utf-8" et al.
print(f"{filename} : file read into a pandas dataframe.")
df

equalratio.csv : file read into a pandas dataframe.


,Unnamed: 0,angle,velocity,netDistanceFromBucket,outcome
0,0,299,16,80.885080,0
1,1,72,29,63.130240,0
2,2,275,3,136.772896,0
3,3,150,6,154.265421,0
4,4,189,39,95.687192,0
...,...,...,...,...,...
47905,47905,138,35,13.376796,1
47906,47906,114,41,46.652239,1
47907,47907,203,46,7.694657,1
47908,47908,201,46,8.660063,1


In [72]:
#
# let's keep our column names in variables, for reference
#
df_clean = df
COLUMNS = df_clean.columns            # "list" of columns
print(f"COLUMNS: {COLUMNS}")  

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX: {COL_INDEX}")

# and for our "SPECIES"!
SPECIES = [ str(i) for i in range(0,10) ]  # list with a string at each index (index -> string)
SPECIES_INDEX = { s:int(s) for s in SPECIES }  # dictionary mapping from string -> index

# and our "target labels"
print(f"SPECIES: {SPECIES}")  
print(f"SPECIES_INDEX: {SPECIES_INDEX}")

COLUMNS: Index(['Unnamed: 0', 'angle', 'velocity', 'netDistanceFromBucket', 'outcome'], dtype='object')
COL_INDEX: {'Unnamed: 0': 0, 'angle': 1, 'velocity': 2, 'netDistanceFromBucket': 3, 'outcome': 4}
SPECIES: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
SPECIES_INDEX: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [73]:
#
# let's convert our dataframe to a numpy array, named A
#    Our ML library, scikit-learn operates entirely on numpy arrays.
#
A = df_clean.to_numpy()    # .values gets the numpy array
A = A.astype('float64')  # so many:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(f"A's shape is {A.shape}")
print(A)

A's shape is (47910, 5)
[[0.00000000e+00 2.99000000e+02 1.60000000e+01 8.08850796e+01
  0.00000000e+00]
 [1.00000000e+00 7.20000000e+01 2.90000000e+01 6.31302401e+01
  0.00000000e+00]
 [2.00000000e+00 2.75000000e+02 3.00000000e+00 1.36772896e+02
  0.00000000e+00]
 ...
 [4.79070000e+04 2.03000000e+02 4.60000000e+01 7.69465687e+00
  1.00000000e+00]
 [4.79080000e+04 2.01000000e+02 4.60000000e+01 8.66006280e+00
  1.00000000e+00]
 [4.79090000e+04 2.89000000e+02 3.70000000e+01 2.45330249e+01
  1.00000000e+00]]


<br>

#### Now, to the bucket-hallucinating part!

In [74]:
#
# We will explore a different direction: "hallucinating" new data!
#      This is sometimes called "imputing" missing data.
#

# specifically, we will build a classifier that
#      + uses the distance from the bucket, angle, and velocity to predict if it went in or not
#      + we'll see how accurate it is...

In [75]:
#
# regression model that uses as input the angle, shot, and distance from bucket to predict if it went in
#

print("+++ Start of classifier prediction of make or miss! +++\n")

X_all = A[:,[1,2]]  ### old: np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # horizontal concatenation
y_all = A[:,4]    # y (labels) ... is all rows, column indexed 52 (pix52) only (actually the 53rd pixel, but ok)

print(f"y_all (just target values, make (1) or miss (0))   is \n {y_all}") 
print(f"X_all (just features: 3 rows) is \n {X_all[:3,:]}")

+++ Start of classifier prediction of make or miss! +++

y_all (just target values, make (1) or miss (0))   is 
 [0. 0. 0. ... 1. 1. 1.]
X_all (just features: 3 rows) is 
 [[299.  16.]
 [ 72.  29.]
 [275.   3.]]


In [76]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("labels (target)\n",y_all)
print("features\n", X_all[:3,:])

labels (target)
 [1. 1. 0. ... 1. 0. 1.]
features
 [[298.  31.]
 [155.  45.]
 [ 41.  13.]]


In [77]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 38328 rows;  testing with 9582 rows

Held-out data... (testing data: 9582)
y_test: [0. 0. 0. ... 0. 1. 1.]

X_test (few rows): [[121.  16.]
 [226.   7.]
 [211.  25.]
 [215.  26.]
 [198.   7.]]

Data used for modeling... (training data: 38328)
y_train: [1. 1. 1. ... 1. 1. 1.]

X_train (few rows): [[226.  48.]
 [208.  48.]
 [225.  32.]
 [ 90.  38.]
 [201.  24.]]


In [78]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
# 
USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    from sklearn.preprocessing import StandardScaler
    scaler_class = StandardScaler()
    scaler_class.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler_class = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler_class.fit(X_train)  # still need to fit, though it does not change...

scaler_class   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here is a fully-scaled dataset:

X_all_scaled = scaler_class.transform(X_all)
y_all_scaled = y_all.copy()      # not scaled


# Here are our scaled training and testing sets:

X_train_scaled = scaler_class.transform(X_train) # scale!
X_test_scaled = scaler_class.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

def ascii_table(X,y):
    """ print a table of binary inputs and outputs """
    print(f"{'input ':>70s} -> {'pred':<5s} {'des.':<5s}") 
    for i in range(len(y)):
        s_to_show = str(X[i,:])
        s_to_show = s_to_show[0:60]
        print(f"{s_to_show!s:>70s} -> {'?':<5s} {y[i]:<5.0f}")   # !s is str ...
    
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5])

#
# Note that the zeros have become -1's
# and the 1's have stayed 1's
#

                                                                input  -> pred  des. 
                                               [0.44146134 1.26474615] -> ?     1    
                                               [0.26859098 1.26474615] -> ?     1    
                                               [0.43185743 0.07042846] -> ?     1    
                                             [-0.8646703   0.51829759] -> ?     0    
                                             [ 0.20136361 -0.52673039] -> ?     0    


In [79]:
from sklearn.neural_network import MLPClassifier

#
# Here's where you can change the number of hidden layers
# and number of neurons!
#
nn_classifier = MLPClassifier(hidden_layer_sizes=(6,7),  # 3 input -> 6 -> 7 -> 1 output
                    max_iter=500,      # how many times to train
                    activation="tanh", # the "activation function" input -> output
                    solver='sgd',      # the algorithm for optimizing weights
                    verbose=True,      # False to "mute" the training
                    shuffle=True,      # reshuffle the training epochs?
                    random_state=None, # set for reproduceability
                    learning_rate_init=.1,       # learning rate: % of error to backprop
                    learning_rate = 'adaptive')  # soften feedback as it converges

# documentation:
# scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html 
#     Try verbose / activation "relu" / other network sizes ...

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_classifier.fit(X_train_scaled, y_train_scaled)
print("\n++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The analog prediction error (the loss) is {nn_classifier.loss_}")



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 0.47152013
Iteration 2, loss = 0.45923097
Iteration 3, loss = 0.45755050
Iteration 4, loss = 0.45556126
Iteration 5, loss = 0.45246639
Iteration 6, loss = 0.44193291
Iteration 7, loss = 0.42541110
Iteration 8, loss = 0.41448929
Iteration 9, loss = 0.40620206
Iteration 10, loss = 0.40086594
Iteration 11, loss = 0.39588928
Iteration 12, loss = 0.39321805
Iteration 13, loss = 0.39101449
Iteration 14, loss = 0.38834554
Iteration 15, loss = 0.38316521
Iteration 16, loss = 0.37646842
Iteration 17, loss = 0.37033478
Iteration 18, loss = 0.36639049
Iteration 19, loss = 0.36098781
Iteration 20, loss = 0.34943251
Iteration 21, loss = 0.34193044
Iteration 22, loss = 0.33895849
Iteration 23, loss = 0.33575665
Iteration 24, loss = 0.33372969
Iteration 25, loss = 0.33239561
Iteration 26, loss = 0.33065133
Iteration 27, loss = 0.32936851
Iteration 28, loss = 0.32741547
Iteration 29, loss = 0.32719615
Iteration 30, loss = 0.32655161

In [80]:
#
# how did it do on the testing data?
#

#
# which one do we want: classifier or regressor?
#

def ascii_table_for_classifier(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc)            # all predictions
    prediction_probs = nn.predict_proba(Xsc) # all prediction probabilities
    Xpr = scaler.inverse_transform(Xsc)      # Xpr is the "X to print": unscaled data!
    # count correct
    num_correct = 0
    # printing
    print(f"{'input ':>28s} -> {'pred':^6s} {'des.':^6s}") 
    for i in range(len(y)):
        pred = predictions[i]
        pred_probs = prediction_probs[i,:]
        desired = y[i]
        if pred != desired: result = "  incorrect: " + str(pred_probs)
        else: result = "  correct"; num_correct += 1
        # Xpr = Xsc  # if you want to see the scaled versions
        print(f"{Xpr[i,:]!s:>28s} -> {pred:^6.0f} {desired:^6.0f} {result:^10s}") 
    print(f"\ncorrect predictions: {num_correct} out of {len(y)}")
    


#
# let's see how it did on the test data (also the training data!)
#
'''
ascii_table_for_classifier(X_test_scaled,
                           y_test_scaled,
                           nn_classifier,
                           scaler) 
                           '''  

                      input  ->  pred   des. 
 [120.66795453  15.98577591] ->   0      0      correct 
 [225.69921563   6.9907013 ] ->   0      0      correct 
 [210.69474976  24.98085051] ->   0      0      correct 
 [214.69594066  25.98030325] ->   0      0      correct 
 [197.69087933   6.9907013 ] ->   0      0      correct 
   [57.64919786 31.97701965] ->   1      1      correct 
 [293.71946091  39.97264152] ->   1      1      correct 
[ 5.06471138e+01 -5.46783235e-03] ->   0      0      correct 
   [99.6617023   4.99179584] ->   0      0      correct 
 [322.72809493  41.97154699] ->   1      1      correct 
   [44.64532744 19.98358685] ->   0      0      correct 
 [128.67033632  46.96881066] ->   1      1      correct 
 [305.72303361  29.97811418] ->   1      1      correct 
 [262.71023145   5.99124857] ->   0      0      correct 
 [327.72958356   4.99179584] ->   0      0      correct 
 [237.70278833  26.97975598] ->   1      1      correct 
   [91.65932051 12.98741771] ->   0  

In [81]:
#
# we have a predictive model!  Let's try it out...
#

def make_prediction( Features, nn, scaler ):
    """ uses nn for predictions """
    print("input features are", Features)
    #  we make sure Features has the right shape (list-of-lists)
    row = np.array( [Features] )  # makes an array-row
    row = scaler.transform(row)   # scale according to scaler
   
    prediction = nn.predict(row)  # max!
    return prediction
    


In [82]:
# PREDICTING ANGLE
# should be columns = ['angle', 'velocity', 'output']

# Build regressor that predicts angle from velocity for a make
filename = 'equalratio.csv'
df = pd.read_csv(filename, header=0)   # encoding="utf-8" et al.
print(f"{filename} : file read into a pandas dataframe.")

A = df.to_numpy()    # .values gets the numpy array
A = A.astype('float64')

#print("+++ Start of regression prediction of angle from starting velocity and output! +++\n")
X_all = A[:,[2,4]]  # velocity and output
y_all = A[:,1]  # angle

print(f"y_all (just target values, angles)   is \n {y_all}") 
print(f"X_all (just features (velocity and output): 3 rows) is \n {X_all}")
print(y_all)
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("labels (target)\n",y_all)
print("features\n", X_all[:3,:])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)


USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    from sklearn.preprocessing import StandardScaler
    scaler_ang = StandardScaler()
    scaler_ang.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler_ang = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler_ang.fit(X_train)  # still need to fit, though it does not change...

scaler_ang   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

X_all_scaled = scaler_ang.transform(X_all)
y_all_scaled = y_all.copy()      # not scaled


# Here are our scaled training and testing sets:

X_train_scaled = scaler_ang.transform(X_train) # scale!
X_test_scaled = scaler_ang.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

from sklearn.neural_network import MLPRegressor

nn_regressor_predict_angle = MLPRegressor(hidden_layer_sizes=(10,10), 
                    max_iter=200,          # how many training epochs
                    activation="tanh",     # the activation function
                    solver='sgd',          # the optimizer
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.01, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor_predict_angle.fit(X_train, y_train)
print("++++++++++  TRAINING:   end  +++++++++++++++")

print(f"The (squared) prediction error (the loss) is {nn_regressor_predict_angle.loss_}")
print(f"And, its square root: {nn_regressor_predict_angle.loss_ ** 0.5}")


equalratio.csv : file read into a pandas dataframe.
y_all (just target values, angles)   is 
 [299.  72. 275. ... 203. 201. 289.]
X_all (just features (velocity and output): 3 rows) is 
 [[16.  0.]
 [29.  0.]
 [ 3.  0.]
 ...
 [46.  1.]
 [46.  1.]
 [37.  1.]]
[299.  72. 275. ... 203. 201. 289.]
labels (target)
 [320.  21. 284. ... 183. 313. 288.]
features
 [[39.  1.]
 [21.  0.]
 [48.  1.]]


++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 5751.33857965
Iteration 2, loss = 5433.41744562
Iteration 3, loss = 5431.30085110
Iteration 4, loss = 5434.99770438
Iteration 5, loss = 5432.64943611
Iteration 6, loss = 5432.21997761
Iteration 7, loss = 5434.71520917
Iteration 8, loss = 5430.40421256
Iteration 9, loss = 5434.15481894
Iteration 10, loss = 5431.97445277
Iteration 11, loss = 5435.17217130
Iteration 12, loss = 5435.16455829
Iteration 13, loss = 5434.31011913
Iteration 14, loss = 5434.14019700
Iteration 15, loss = 5435.16238280
Iteration 16, loss = 5436.38019110
Iterati

In [83]:
# PREDICTING VELOCITY
# should be columns = ['angle', 'velocity', 'output']

# Build regressor that predicts angle from velocity for a make
filename = 'equalratio.csv'
df = pd.read_csv(filename, header=0)   # encoding="utf-8" et al.
print(f"{filename} : file read into a pandas dataframe.")

A = df.to_numpy()    # .values gets the numpy array
A = A.astype('float64')

#print("+++ Start of regression prediction of angle from starting velocity and output! +++\n")
X_all = A[:,[1,4]]  # angle and output
y_all = A[:,2]  # velocity

print(f"y_all (just target values, velocity)   is \n {y_all}") 
print(f"X_all (just features (velocity and output): 3 rows) is \n {X_all}")
print(y_all)
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("labels (target)\n",y_all)
print("features\n", X_all[:3,:])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)


USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    from sklearn.preprocessing import StandardScaler
    scaler_vel = StandardScaler()
    scaler_vel.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler_vel = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler_vel.fit(X_train)  # still need to fit, though it does not change...

scaler_vel   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

X_all_scaled = scaler_vel.transform(X_all)
y_all_scaled = y_all.copy()      # not scaled


# Here are our scaled training and testing sets:

X_train_scaled = scaler_vel.transform(X_train) # scale!
X_test_scaled = scaler_vel.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

from sklearn.neural_network import MLPRegressor

nn_regressor_predict_velocity = MLPRegressor(hidden_layer_sizes=(10,10), 
                    max_iter=200,          # how many training epochs
                    activation="tanh",     # the activation function
                    solver='sgd',          # the optimizer
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.01, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor_predict_velocity.fit(X_train, y_train)
print("++++++++++  TRAINING:   end  +++++++++++++++")

print(f"The (squared) prediction error (the loss) is {nn_regressor_predict_velocity.loss_}")
print(f"And, its square root: {nn_regressor_predict_velocity.loss_ ** 0.5}")


equalratio.csv : file read into a pandas dataframe.
y_all (just target values, velocity)   is 
 [16. 29.  3. ... 46. 46. 37.]
X_all (just features (velocity and output): 3 rows) is 
 [[299.   0.]
 [ 72.   0.]
 [275.   0.]
 ...
 [203.   1.]
 [201.   1.]
 [289.   1.]]
[16. 29.  3. ... 46. 46. 37.]
labels (target)
 [37. 16. 45. ... 42. 39. 12.]
features
 [[134.   1.]
 [246.   0.]
 [155.   1.]]


++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 97.16582396
Iteration 2, loss = 89.78860485
Iteration 3, loss = 89.76476102
Iteration 4, loss = 89.73028308
Iteration 5, loss = 89.78325659
Iteration 6, loss = 89.75777841
Iteration 7, loss = 89.78463724
Iteration 8, loss = 89.74272113
Iteration 9, loss = 89.75435638
Iteration 10, loss = 89.73693930
Iteration 11, loss = 89.74825912
Iteration 12, loss = 89.71757874
Iteration 13, loss = 89.82155610
Iteration 14, loss = 89.75625041
Iteration 15, loss = 89.74639057
Iteration 16, loss = 89.76279986
Iteration 17, loss = 89.77790317
Iter

In [94]:
#scaler = scaler_vel
def predict_from_model(pixels, model, customscaler):
    """ returns the prediction on the input pixels using the input model
    """
    pixels_array = np.asarray([pixels])   # the extra sq. brackets are needed!
    pixels_scaled = customscaler.transform(pixels_array)  # need to use the scaler!
    predicted = model.predict(pixels_scaled)
    return predicted

print(make_prediction([10, 1], nn_regressor_predict_angle, scaler_ang))

input features are [10, 1]
[-112.61209551]


In [106]:
starting_angle = 145
make_miss = ['Miss.', 'Make!']

print("Predicting velocity for an angle of " + str(starting_angle) + " degrees...")
for vel in range(0,1000):
    if str(predict_from_model([starting_angle,vel], nn_classifier, scaler_class)) == '[1.]':
        print("Result of a shot with angle of " + str(starting_angle) + " degrees and a velocity of " + str(vel) + ": Make!")
    elif str(predict_from_model([starting_angle,vel], nn_classifier, scaler_class)) == '[0.]':
        print("Result of a shot with angle of " + str(starting_angle) + " degrees and a velocity of " + str(vel) + ": Miss.")

Predicting velocity for an angle of 145 degrees...
Result of a shot with angle of 145 degrees and a velocity of 0: Miss.
Result of a shot with angle of 145 degrees and a velocity of 1: Miss.
Result of a shot with angle of 145 degrees and a velocity of 2: Miss.
Result of a shot with angle of 145 degrees and a velocity of 3: Miss.
Result of a shot with angle of 145 degrees and a velocity of 4: Miss.
Result of a shot with angle of 145 degrees and a velocity of 5: Miss.
Result of a shot with angle of 145 degrees and a velocity of 6: Miss.
Result of a shot with angle of 145 degrees and a velocity of 7: Miss.
Result of a shot with angle of 145 degrees and a velocity of 8: Miss.
Result of a shot with angle of 145 degrees and a velocity of 9: Miss.
Result of a shot with angle of 145 degrees and a velocity of 10: Miss.
Result of a shot with angle of 145 degrees and a velocity of 11: Miss.
Result of a shot with angle of 145 degrees and a velocity of 12: Miss.
Result of a shot with angle of 145 d